In [10]:
import cv2
import numpy as np
from scipy.ndimage import convolve


def blur_img(image, value=15):
    return cv2.GaussianBlur(image,(value,value), 0) # most effective in the paper was 12,12, but that's a heavy blur

def brightness_img(image, value=125):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:,:,2] = cv2.add(hsv[:,:,2], value)
    image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return image

def nodemosaic_img(image):
    w, h, _ = image.shape
    result = np.zeros((w, h, 3), dtype=np.uint8)        # make a target array, which will be colored with the blue, green, and red values as if it has not been demosaiced
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    result[::2, ::2, 2] = image[::2, ::2, 2]       # blue
    result[1::2, ::2, 1] = image[1::2, ::2, 1]     # green
    result[::2, 1::2, 1] = image[::2, 1::2, 1]     # green
    result[1::2, 1::2, 0] = image[1::2, 1::2, 0]   # red
    return result

def demosaic(image):
    h, w = image.shape[:2]
    result = np.zeros((h, w, 3), dtype=np.uint8)
    
    kernel_cross = np.array([[0, 1, 0],
                           [1, 0, 1],
                           [0, 1, 0]]) / 6
    
    kernel_diagonal = np.array([[1, 0, 1],
                              [0, 0, 0],
                              [1, 0, 1]]) / 4
    
    R = image[..., 0]
    G = image[..., 1]
    B = image[..., 2]
    
    red_mask = R > 1
    blue_mask = B > 1
    green_mask = G > 1
       
    result[..., 0] = np.where(red_mask, R,
                             np.where(green_mask,
                                     convolve(R, kernel_cross),
                                     convolve(R, kernel_diagonal)))
    
    result[..., 1] = np.where(green_mask, G,
                             convolve(G, kernel_cross))
    
    result[..., 2] = np.where(blue_mask, B,
                             np.where(green_mask,
                                     convolve(B, kernel_cross),
                                     convolve(B, kernel_diagonal)))
    
    return result

image = cv2.imread('driving_image.jpeg')
image = cv2.resize(image, (640, 360))
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
blurred = blur_img(image)
bright = brightness_img(image)
nodemosaic = nodemosaic_img(image)
demosaic = demosaic(nodemosaic)
# nodemosaic = np.uint16(nodemosaic)
# demosaic = cv2.demosaicing(nodemosaic, cv2.COLOR_BayerGB2RGB )
cv2.imwrite('image.jpg', image)
cv2.imwrite('blurred.jpg', blurred)
cv2.imwrite('bright.jpg', bright)
cv2.imwrite('nodemosaic.jpg', nodemosaic)
cv2.imwrite('demosa.jpg', demosaic)
# cv2.imwrite('demosaic.jpg', demosaic)


True

In [46]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# 
# 
# 
# def plot_image_and_spectrum(img1, title1="Image 1", vmin=None, vmax=None):
#     # Set consistent figure size
#     plt.figure(figsize=(6, 5))
#     
#     spectrum1 = compute_magnitude_spectrum(img1)
#     plt.imshow(spectrum1, cmap='gray', vmin=vmin, vmax=vmax)
#     plt.title(f'Magnitude Spectrum of {title1}')
#     plt.colorbar()
#     
#     plt.show()
# 
# # Read the images

# 
# # Compute both spectrums first to get global min and max
# spectrum1 = compute_magnitude_spectrum(img_no_demosaic)
# spectrum2 = compute_magnitude_spectrum(img)
# 
# # Find global min and max for consistent colorbar scaling
# vmin = min(spectrum1.min(), spectrum2.min())
# vmax = max(spectrum1.max(), spectrum2.max())
# 
# # Plot the images and their magnitude spectrums
# plot_image_and_spectrum(img_no_demosaic, 
#                        title1="Image - not Demosaiced",
#                        vmin=vmin, vmax=vmax)
# plot_image_and_spectrum(img, 
#                        title1="Image - Demosaiced",
#                        vmin=vmin, vmax=vmax)

In [49]:
img_no_demosaic = cv2.imread('nodemosaic.jpg')
img = cv2.imread('driving_image.jpeg')

def compute_magnitude_spectrum(image):
    # Convert to grayscale if the image is in color
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1)
    return magnitude_spectrum

def analyze_frequency_pattern(magnitude_spectrum):
    h, w = magnitude_spectrum.shape
    center_h, center_w = h//2, w//2
    
    low_freq_mask = np.zeros_like(magnitude_spectrum)
    low_freq_mask[center_h-10:center_h+10, center_w-10:center_w+10] = 1
    
    high_freq_mask = np.ones_like(magnitude_spectrum)
    high_freq_mask[center_h-50:center_h+50, center_w-50:center_w+50] = 0
    
    low_freq_energy = np.sum(magnitude_spectrum * low_freq_mask)
    high_freq_energy = np.sum(magnitude_spectrum * high_freq_mask)
    
    return high_freq_energy / low_freq_energy

def has_demosaic_failure(input_image):
    return analyze_frequency_pattern(compute_magnitude_spectrum(input_image)) < 750


print(has_demosaic_failure(img_no_demosaic))
print(has_demosaic_failure(img))


True
False
